## Import library

In [15]:
# Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
# Goal is to predict if protein is a SNP or PD
#ImprovedBalancing branch

#Imports the required libraries and packages
import pandas as pd  #Import for data manipulation in dataframes
import numpy as np  #Array manipulation and calculates mean
import matplotlib.pyplot as plt  #Graphing and plotting
# show figures in jupyter instead of prompt window
%matplotlib inline 

import random as rd
import time
import sys

from sklearn import tree

from sklearn.metrics import(
    matthews_corrcoef,  # CC for evaluation
    f1_score,  #F1 score for evaluation
    balanced_accuracy_score, roc_auc_score, make_scorer,  #Scoring metrics
    confusion_matrix,  #Creates the confusion matrix - stats on how accurate the test set output is
    classification_report #Returns the F1 socre, precision, and recall of a prediction using a given model
    )
from sklearn.model_selection import(
    train_test_split,  # Splits data frame into the training set and testing set
    GridSearchCV,  # Cross validation to improve hyperparameters
    StratifiedKFold
        )
from sklearn.ensemble import RandomForestClassifier #SK learn API for classificastion random forests
from sklearn.tree import DecisionTreeClassifier #Single tree decisions 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle #shuffles rows
from sklearn.neighbors import KNeighborsClassifier #allows for confidence scores to be predicted for each

np.set_printoptions(threshold=np.inf) #full array printing

## Random Seed function

In [16]:
def Random_Seed(): #Generates a random seed
    random1 = rd.randrange(1,100) #Random integet between 1 and 100
    random2 =  time.time() #Time since UTC epoch
    Seed = int(random2//random1//1000)
    return Seed
Random_Seed()

75931

## Read the whole dataset - revis

In [79]:
#Create, clean and convert dataset E2.csv to PD dataframe**
df = pd.read_csv('E2.csv')  #Create PD data frame from .csv
df.drop(['pdbcode:chain:resnum:mutation'], axis=1, inplace=True)  #Removes unrequired columns. PDBcode may be needed for manual validation 
df.columns = df.columns.str.replace(' ', '_')  # Removes any blank attributes
df.replace(' ', '_', regex=True, inplace=True)  # Replace all blank spaces with underscore (none were present)
df.reset_index(drop=True, inplace = True) #Resets index numbering from 0 and drops column
Input = df.drop('dataset', axis =1).fillna('0') #DF of input instances for classification training. Unknown attributes assigned 0
Output_encoded = pd.get_dummies(df, columns=['dataset']) #One hot encoding dataset column so "PD" and "SNP" attributes are numerical 0 or 1
Output = Output_encoded['dataset_pd'].copy().astype('int32') #Dataframe with 1 column where 1 = PD, 0 = SNP, integer

print("Total samples:", len(df))
Majority = len(df.loc[df['dataset'] == 'pd'])
print(f"{Majority} PD samples")
Minority = len(df.loc[df['dataset'] == 'snp'])
print(f"{Minority} PD samples")

Total samples: 3368
2254 PD samples
1111 PD samples


## Split into training and testing, generate RF (whole dataset)

In [80]:
Input_train, Input_test, Classes_train, Classes_test = train_test_split(Input, Output, train_size = 0.8, random_state=42, stratify=Output) #80% training and 20% testing split. Strartify ensures fixed poportion of output labels is in both sets. Input attributes and class labels, training attributes and class label etc
start=time.time() #Start timer for inital training model building
RFC = RandomForestClassifier(random_state = 42, n_estimators = 1000, verbose = 1) #Defines the Random Forest. 42 seeds, 1000 trees
RFC.fit(Input_train, Classes_train) #Generates a random forest from the training data

with open('Training Data.txt', 'w') as file: #Writes class labels for all instances to text file
    file.write(Input_train.to_string())
    
with open('Class labels.txt', 'w') as file: #Writes class labels for all instances to text file
    file.write(Output.to_string())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    4.4s finished


## Training (revisit params)

In [81]:
# StandardScaler().fit(X_train).transform(X_train) #Scales data 
# pipeline = make_pipeline( #Sets the random forest parameters
#     StandardScaler(),
#     LogisticRegression(solver='saga', max_iter=2000),
#     verbose=2
# )
RFC.get_params()
# Evaluation of training before weighted vote

Output_pred = RFC.predict(Input_test) #Always perdict on the unseen test data, as train has been used by the estimastor
print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
print(f"MCC:\n {matthews_corrcoef(Classes_test, Output_pred)}")
print("F1:\n", f1_score(Classes_test, Output_pred))
print(classification_report(Classes_test, Output_pred))

Confusion Matrix:
 [[145  78]
 [ 27 424]]
MCC:
 0.6371468255225344
F1:
 0.8898216159496328
              precision    recall  f1-score   support

           0       0.84      0.65      0.73       223
           1       0.84      0.94      0.89       451

    accuracy                           0.84       674
   macro avg       0.84      0.80      0.81       674
weighted avg       0.84      0.84      0.84       674



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


# Weighted Voting

### Find majority:minority ratio and number of balancing folds

In [82]:
def Ratio_divide(): #Finds ratio between the 2 classes (i.e the imbalance) and the number of folds required
    a = Majority
    b = Minority
    Divide = a/b
    if Divide <= 1:
        Ratio = 1/Divide
    else:
        Ratio = Divide
        
    if round(Ratio) % 2 == 0:
        BF = 2 * round(Ratio) + 1
    else:
        BF = round(Ratio)
    return Ratio, BF;
Ratio, BF = Ratio_divide()
print(f"Imbalance ratio:\n{Ratio}:1\n{BF} balancing folds needed.")

Imbalance ratio:
2.028802880288029:1
5 balancing folds needed.


## Balancing via array index

In [83]:
def Balancing_Fold():
    Output_reset = Output.reset_index() #resets index and drops an un-needed "index" column
    Output_Array = np.array(Output_reset) #Array of class labels. 1 = PD = majority, 0 = SNP = minority
    Array = np.squeeze(Output_Array) #Makes array 1D

    count_0 = 0
    count_1 = 0
    list_index = []
    Array_loop = shuffle(Array)

    for i in Array_loop:
        index = int(i[0:1])
        values = i[1:]

        if values == 0 and count_0 < Minority:
            count_0 += 1
            list_index.append(index)
        elif values == 1 and count_1 < Minority:
            count_1 += 1
            list_index.append(index)
#     print(f"PD:{count_1}\nSNP:{count_0}\nTotal:{count_1+count_0}")

    Balanced_Input = df.filter(list_index, axis = 0)
        
    return Balanced_Input

In [84]:
def Clean_Balance_Fold():
    BF_List = []
    for i in range(BF):
        BF_List.append(Balancing_Fold())
    return BF_List
Folds = Clean_Balance_Fold()

In [159]:
Folds

[      Binding  SProtFT0  SProtFT1  SProtFT2  SProtFT3  SProtFT4  SProtFT5  \
 67        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 356       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 2740      0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 139       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 912       1.0       0.0       0.0       0.0       0.0       0.0       0.0   
 ...       ...       ...       ...       ...       ...       ...       ...   
 140       0.0       0.0       0.0       0.0       0.0       1.0       0.0   
 1205      0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 2796      0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 1042      0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 1404      0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 
       SProtFT6  SProtFT7  SProtFT8  ...  NLargest6  NLargest7

## Split into training and testing, generate RF (balanced dataset)

In [161]:
def BF_training():
    Input_list = []
    Output_list=[]
    for i in range(BF):
        BF_Input = Folds[i].drop('dataset', axis = 1).fillna('0')
        Input_list.append(BF_Input)
        
        BF_encoded = pd.get_dummies(Folds[i], columns=['dataset']) #One hot encoding dataset column so "PD" and "SNP" attributes are numerical 0 or 1
        BF_Output = BF_encoded['dataset_pd'].copy().astype('int32') #Dataframe with 1 column where 1 = PD, 0 = SNP, integer
        Output_list.append(BF_Output)
        
    return Input_list, Output_list
Input_balanced, Output_balanced = BF_training()
        
a = 0
print(Input_balanced[a])
print(Output_balanced[a])
        
# Input_train, Input_test, Classes_train, Classes_test = train_test_split(Input, Output, train_size = 0.8, random_state=42, stratify=Output) #80% training and 20% testing split. Strartify ensures fixed poportion of output labels is in both sets. Input attributes and class labels, training attributes and class label etc

# BF_RFC = RandomForestClassifier(random_state = 42, n_estimators = 1000, verbose = 1) #Defines the Random Forest. 42 seeds, 1000 trees
# RFC.fit(Input_train, Classes_train) #Generates a random forest from the training data

# with open('Training Data.txt', 'w') as file: #Writes class labels for all instances to text file
#     file.write(Input_train.to_string())
    
# with open('Class labels.txt', 'w') as file: #Writes class labels for all instances to text file
#     file.write(Output.to_string())

     Binding SProtFT0 SProtFT1 SProtFT2 SProtFT3 SProtFT4 SProtFT5 SProtFT6  \
67       0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
356      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2740     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
139      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
912      1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
...      ...      ...      ...      ...      ...      ...      ...      ...   
140      0.0      0.0      0.0      0.0      0.0      1.0      0.0      0.0   
1205     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2796     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1042     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1404     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

     SProtFT7 SProtFT8  ... NLargest5 NLargest6 NLa

In [ ]:
#Build a model for each balancing fold, predict the confidence scores and find average for each, which gives final vote
 
BF_RFC = RandomForestClassifier(random_state = 42, n_estimators = 100) #Defines the Random Forest. 42 seeds, 100 trees (increase)
BF_Prob = []
BF_Probtxt = []
BF_data = []

#Randomly samples the majority class PD, to the same size of minority class SNP, and scores confidence for each instance
for i in range (BF):
    a = Random_Seed()
    BF_Input_PD_train = Input_PD_train.sample(frac = (1/Ratio), random_state = a//100) #Balanced PD input training data 
    BF_Classes_PD_train = Classes_PD_train.sample(frac = (1/Ratio), random_state = a//100) #Balanced PD output training data
    
    #Concatanates the input and output balancing fold training data, so a new RFC can be generated. Function
    BF_Input_all = shuffle(pd.concat([BF_Input_PD_train, Input_SNP_train]), random_state = a//50)
    BF_Output_all = shuffle(pd.concat([BF_Classes_PD_train, Classes_SNP_train]), random_state = a//50)
    Full_set = pd.concat([BF_Input_all, BF_Output_all], axis = 1)


    #Generates RF and calculates probabilities for each fold
    BF_RFC.fit(BF_Input_all, BF_Output_all)
    BF_data.append(Full_set) #Builds random forest from training data for each fold
    
    Prob_index = BF_RFC.predict_proba(BF_Input_all) #Predicts confidence score for each instance
    Full_set[['SNP', 'PD']] = Prob_index
    Prob_index = Full_set.drop(labels=['Binding','SProtFT0','SProtFT1','SProtFT2','SProtFT3','SProtFT4','SProtFT5','SProtFT6','SProtFT7','SProtFT8','SProtFT9','SProtFT10','SProtFT11','SProtFT12','Interface','Relaccess','Impact','HBonds','SPhobic','CPhilic','BCharge','SSGeom','Voids','MLargest1','MLargest2','MLargest3','MLargest4','MLargest5','MLargest6','MLargest7','MLargest8','MLargest9','MLargest10','NLargest1','NLargest2','NLargest3','NLargest4','NLargest5','NLargest6','NLargest7','NLargest8','NLargest9','NLargest10','Clash','Glycine','Proline','CisPro','dataset_pd'], axis=1, inplace=False)
    #Becomes a list
    BF_Prob.append(Prob_index) #List with probabilites for all instances
    BF_Probtxt.append(Prob_index.to_string())
    
with open('Balanced probabilities.txt', 'w') as f:
    for number, line in zip(range(BF), BF_Probtxt):
        f.write(f"Fold: {number}\n\n{line}\n\n\n")
        
with open('Balanced training data.txt', 'w') as f:
    for number, fold in zip(range(BF), BF_data):
        f.write(f"Fold: {number}\n\n{fold}\n\n\n")

### Weighted voting for each class

In [84]:
#Apply weighted vote scheme for a predictor that outputs confidence value between 0 and 1 for each class. 
def Weighted_Proba(BF_Prob, BF_list):

    # Identify which instance appear in all folds, uding the nth fold (random number)
    intersect_index = set(BF_Prob[0].index.values) #index of all 1782 instances in first fold
    
    for i in BF_list: #for all 5 folds
        intersect_index = intersect_index.intersection(set(BF_Prob[i].index.values)) #intersection checks if instances are in all folds
    intersect_index_list = list(intersect_index) #converts set to list, 945 items

    with open('All Instances.txt', 'w') as f:
        for line in intersect_index_list:
            f.write(f"{line}\n")
            
    BF_prob_list_PD =[]
    BF_prob_list_SNP =[]
    for i in BF_list: #for all 5 folds
        BF_Prob_instance = BF_Prob[i].loc[intersect_index_list,:] #Returns each dataframe with the common instances
        BF_prob_list_PD.append(BF_Prob_instance.iloc[:,1] - BF_Prob_instance.iloc[:,0]) #PD - SNP prob
        BF_prob_list_SNP.append(BF_Prob_instance.iloc[:,0] - BF_Prob_instance.iloc[:,1]) #SNP - PD prob   
    
    SNP_Sum = 0
    for i in BF_list:
        SNP_Sum += BF_prob_list_SNP[i]
    PD_Sum = 0
    for i in BF_list:
        PD_Sum += BF_prob_list_PD[i]
        
    S_out = abs((PD_Sum - SNP_Sum)/(len(BF_list) * 2))
    return(S_out)

Weighted_Proba(BF_Prob = BF_List, BF_list=range(BF))

2       0.0
5       0.0
2054    0.0
2055    0.0
8       0.0
       ... 
2025    0.0
2027    0.0
2034    0.0
2036    0.0
2039    0.0
Length: 1171, dtype: float64

In [ ]:
Prob_PD = RFC.predict_proba(Input_PD_train) #Probability prediction, as a mean of all trees in RF, for the PD instances
Prob_SNP = RFC.predict_proba(Input_SNP_train) #Probability prediction, as a mean of all trees in RF, for the SNP instances

In [ ]:
S_out_string = Weighted_Proba(BF_Prob, BF_list=range(BF)).to_string() #Write S_out to text file
with open('S_out.txt', 'w') as f:
    f.write(S_out_string)

In [ ]:
BF_Prob[0].iloc[:,0]

In [ ]:
#Final vote

PD = BF_Prob[0].iloc[:,1].to_numpy()
SNP = BF_Prob[0].iloc[:,0].to_numpy()
# if np.greater(PD,SNP):

# PD_Count = 0
# SNP_Count = 0
# for i in range(len(BF_Prob)):
#     if np.greater(PD,SNP):
#         PD_Count += 1
#     elif BF_Prob[i].iloc[:,0] > BF_Prob[i].iloc[:,1]:
#         SNP_Count += 1
#     else:
#         pass

        

# PD_Count = 0
# for i in Weighted_Proba(BF_Prob, BF_list=range(BF)):
#     if i > 0.5:
#         PD_Count += 1
#     else:
#         pass
# print(PD_Count,"samples predicted to be PD")

# SNP_Count = 0
# for i in Weighted_Proba(BF_Prob, BF_list=range(BF)):
#     if i < 0.5:
#         SNP_Count += 1
#     else:
#         pass
# print(SNP_Count,"samples predicted to be SNP")


# #Evaluation of training after weighted vote
# Classes_pred = RFC.predict(Input_test)
# print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Classes_pred)}")
# print(f"MCC:\n {matthews_corrcoef(Classes_test, Classes_pred)}")
# print("F1:\n", f1_score(Classes_test, Classes_pred))

In [ ]:
# TODO: retrive the probability from each tree for a single sample  

### Validation

In [ ]:
# # **Split data into training and test**
# with open('SNPorPD.txt', 'w+') as f:
#         data=f.read()
#         f.write(str(y_test.to_string()))

# # pipeline.fit(X, y) #applies list if transformers to give a fitted model

plt.scatter(Classes_test, Output_pred)

In [ ]:
gridsearch = GridSearchCV( #validation
    estimator = LogisticRegression(solver='saga'),
    param_grid = {}, #dictionary of parameters to search through
    cv = StratifiedKFold(),
    n_jobs = 1, #how many processors to run in parallel
    scoring = 'f1',
    verbose = 3 
    ).fit(X_train, y_train)

In [ ]:
# y_pred = clf.predict(X_test)
# print("Training time:", stop-start)
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("MCC:\n", matthews_corrcoef(y_test, y_pred))
# print("F1:\n", f1_score(y_test, y_pred))